In [2]:
%pip install azure-storage-blob
%pip install opencv-python

from azure.storage.blob import BlobServiceClient
import json
import os
from PIL import Image
import matplotlib.pyplot as plt
import io  # Used to convert bytes to a file-like object
import cv2
import numpy as np
import pandas as pd

# open CSV
df = pd.read_csv('METER_ML_train.csv')

#set up storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=mlfinalexam5505462853;AccountKey=0c40lghglG5/GlNK9yujDQAgo38GKoS2I3DeC/g22hwAEIFANKpmC/TqOpRk4RCT1DbfNiHBFt72+AStB+PfUA==;EndpointSuffix=core.windows.net"
container_name = "meterml"

#create client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


FileNotFoundError: [Errno 2] No such file or directory: 'METER_ML_train.csv'

In [5]:
def augment_image(image): 
    # Flip the image horizontally
    flipped_image = cv2.flip(image, 1)

    # Flip the image vertically
    #flipped_image = cv2.flip(image, 0)

    # Rotate the image 90 degrees
    #rotated_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)

    return flipped_image

# Function to upload a jpg file to Azure Blob Storage with a given folder name
def upload_jpg_to_blob(image, file_name, folder_name):
    # Blob name with prefix
    blob_name = f"augmented/{folder_name}/{file_name}"

    # Create a BlobClient for the blob
    blob_client = container_client.get_blob_client(blob_name)
    
    # Encode image to bytes
    success, encoded_image = cv2.imencode('.png', image)
    if not success:
        print(f"Error encoding image: {file_name}")
        return
    
    # Upload the encoded image to Azure Blob Storage
    blob_client.upload_blob(encoded_image.tobytes(), overwrite=True)  # Overwrite if the blob exists

folders = ['test_images','train_images_1','train_images_2','train_images_3','val_images']

# Loop through each folder
for folder_name in folders:
    # Get list of blobs (images) from the folder in Azure Blob Storage
    blobs = container_client.list_blobs(name_starts_with=f"samples/{folder_name}/")

    # Loop through each blob (image) in the folder
    for blob in blobs:
        if blob.name.endswith(".png"):  # Check if the blob is a png image
            # Download the image data
            blob_data = container_client.download_blob(blob).readall()
            
            # Convert the image data to a numpy array
            nparr = np.frombuffer(blob_data, np.uint8)
            
            # Decode the image
            image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            
            # Augment the image
            augmented_image = augment_image(image)
    
            # Upload the preprocessed image to Azure Blob Storage with the correct folder structure
            upload_jpg_to_blob(augmented_image, os.path.basename(blob.name), folder_name)

KeyboardInterrupt: 